# Desafío 01

* Importación de librerías
* astype()

In [ ]:
#!pip install unidecode


In [1]:
import sys
sys.path.append("../src")
import g3utils as g3

import pandas as pd
import numpy as np
import re as re
import unidecode

In [ ]:
# Para ver el contenido total de las columnas

pd.set_option('display.max_colwidth', -1)

In [2]:
df = pd.DataFrame(pd.read_csv("../data/properatti_minusculas.csv", encoding='UTF-8'))
#df = pd.DataFrame(pd.read_csv("../data/properatti.csv", encoding='UTF-8'))

#!head nombreArchivo.cvs --> comando de linux para ver las 5 primeras lineas. Se puede utilizar para ver como estan separados los datos

In [ ]:
df.info(verbose=None, null_counts=True)

In [3]:
df.sample(1)['description']

5822    departamento de 2 ambientes en venta. bano com...
Name: description, dtype: object

* Información de las columnas

In [ ]:
# Busqueda de NaNs
df[pd.isnull(df['description'])]


### Pruebas de campos

In [ ]:
df.iloc[55476]['description']

### Funciones

In [ ]:
# Busca claves segun pattern en columna dentro de dataFrame 
# Entrada: pattern, columna, df
# Salida: DataFrame

def busca_claves(pattern, columna, df_aux):
    regex = re.compile(pattern, flags = re.IGNORECASE)
    m = pd.DataFrame([regex.findall(n) for n in df_aux[columna].head(10000)])
    return m[0]

In [ ]:
def existe_clave(key, columna, df_aux):    
    m = []
    for frase in df_aux[columna]:
        if key in frase:
            m.append(True)
        else:
            m.append(False)
    return sum(m)


In [ ]:
# No se está utilizando mas desde que el dataset se encuentra en minúsculas y sin acentos
def quitar_caracteres(column):
    for i in range(len(column)):
        if type(column[i]) == str:
            column[i] = str.lower(unidecode.unidecode(column[i]))

In [ ]:
# Cambia los literales numericos x floats. En caso de no detectar la clave en un diccionario, reemplaza por NaN.
# El reemplazo x NaN se hace para poder realizar operaciones.
# Mejoras: debiera recibir diccionario y lista_prop como parametros para dejar una función polimorfica. 
# Entrada: DataFrame
# Salida: DataFrame

def cambiar_x_nros(df):
    dic = {'mono': 1, 'un':1, 'uno':1, 'dos':2, 'tres':3, 'cuatro':4, 'cinco':5 ,'seis':6 ,'siete':7}
    lista_prop = ['mono','un','uno','dos','tres','cuatro','cinco','seis','siete']
    serie_1 = pd.Series([x if x not in lista_prop else dic.get(x) for x in df])
    serie_2 = pd.to_numeric(serie_1, errors='coerse', downcast='float') # coerse: pasa a NaN los no-numericos
    return pd.DataFrame(serie_2)

In [ ]:
quitar_caracteres(df["place_name"])
quitar_caracteres(df["place_with_parent_names"])
quitar_caracteres(df["country_name"])
quitar_caracteres(df["state_name"])
quitar_caracteres(df["description"])
quitar_caracteres(df["title"])


### Patterns

In [4]:
# Comentar lo que no se deba ejecutar

#pattern = r'(\d+)\s[m]2' # Patron para buscar metros cuadrados (m2)
pattern = r'(\d*\w*)\s*amb'  # Patron para buscar metros ambientes
#pattern = r'(\d*)\s?\b[aA]mbientes\w*'
#pattern = r'(\d*\w*)\s*usd'
#pattern = r'(\d*\w*)\s*habitacion|(\d*\w*)\s?cuarto|(\d*\w*)\s?dorm|(\d*\w*)\s?pieza'

In [ ]:
#regex = re.compile(pattern, flags = re.IGNORECASE)

### Testing Regex

In [6]:
df1 = g3.busca_claves(pattern,'description',df.head(10000))
#df1 = busca_claves(pattern,'description',df)
#df2 = busca_claves(pattern,'title',df)

### Limpieza de salida de Regex

In [7]:
# Tener en cuenta que se está probando con 10.000 elementos. Se puede modificar dentro de busca_claves()
g3.cambiar_x_nros(df1)


,0
0,2.0
1,NaN
2,2.0
3,3.0
4,NaN
5,NaN
6,NaN
7,1.0
8,2.0
9,NaN


# SIN USO

In [ ]:
#tmp_df = df["cases"].str.extract("([a-z])([0-9]+)([0-9]{2})",expand=True) 
tmp_df = df["description"].str.extract("((\d*)\s?\b[aA]mb\w*)",expand=True) 
tmp_df.dropna(axis=0)

In [ ]:
# TEST: Regex de prueba. Alimenta un DF con el resultado
# CONDICION: No lee NaN's
# NOTA: Me di cuenta que REGEX analiza todo el str y en muchos casos detecta la KEY en varias partes del 
#   str, ergo, al capturar el siguiente dato, lo incluye dentro de una lista y eso lo representa en otra columna
#   Entonces ponerle nombre a una única columna no tendría sentido. Si verificamos con .head(10) no vamos
#   a tener problemas porque justo las 10 primeras filas no tienen mas de una KEY en el str a analizar...

#m = pd.DataFrame([regex.findall(n) for n in df_a1['description'].head(10)], columns=["ambientes"] )
m = pd.DataFrame([regex.findall(n) for n in df_a1['description'].head(50)])
#m = pd.DataFrame([regex.findall(n) for n in df_a1['description']])
display(len(m))
#m
m[0]


### Reemplazo de NaN's
Esto se realiza porque la función de búsqueda explotaba 

In [ ]:
df_a1=df.replace(np.nan,'sin-dato')
#df_a1.head(2)

In [ ]:
# Coloco nombre en la columna para luego hacer merge

df1.rename(columns={0:'ambientes'}, inplace=True)